In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

data_dir = './all_data/'

# Define data augmentation transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to the expected size for ResNet18
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize using ImageNet's mean and std
])

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=64)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=64)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)

# Load pretrained ResNet18 model
model = models.resnet152(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(dataset.classes))  # Modify the final fully connected layer for classification

# Move model to GPU for acceleration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.00001, weight_decay=0.09)

# Train the model
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=100):
    train_accuracies = []
    val_accuracies = []
    train_losses = []
    val_losses = []

    model.train()
    for epoch in range(num_epochs):
        train_loss = 0.0
        correct = 0
        total = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}, Train')
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            progress_bar.set_postfix({'loss': train_loss / len(progress_bar), 'accuracy': 100 * correct / total})

        train_accuracy = 100 * correct / total
        val_loss, val_accuracy = evaluate_model(model, criterion, val_loader)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        train_losses.append(train_loss / len(train_loader))
        val_losses.append(val_loss)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    return train_accuracies, val_accuracies, train_losses, val_losses

def evaluate_model(model, criterion, dataloader):
    model.eval()
    loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss += criterion(outputs, labels).item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = loss / len(dataloader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

# Train the model and save training information

train_accuracies, val_accuracies, train_losses, val_losses = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20)

# Save the model
torch.save(model.state_dict(), "model.pth")
print("Model saved as model.pth")

# Test the model
test_loss, test_accuracy = evaluate_model(model, criterion, test_loader)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

# Save training information to CSV
df = pd.DataFrame({
    'Epoch': range(1, 11),
    'Train Accuracy': train_accuracies,
    'Val Accuracy': val_accuracies,
    'Train Loss': train_losses,
    'Val Loss': val_losses
})
df.to_csv('training_log.csv', index=False)
print("Training log saved as training_log.csv")

# Save accuracy and loss plots as an image
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, 11), train_accuracies, label='Train Accuracy')
plt.plot(range(1, 11), val_accuracies, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, 11), train_losses, label='Train Loss')
plt.plot(range(1, 11), val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss')
plt.legend()

plt.tight_layout()
plt.savefig("training_plot.png")
print("Training plot saved as training_plot.png")


/home/p2/miniconda3/envs/vishw/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/p2/miniconda3/envs/vishw/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cuda:0


Epoch 1/20, Train: 100%|██████████| 1153/1153 [24:47<00:00,  1.29s/it, loss=1.26, accuracy=68.8]


Epoch [1/20], Train Loss: 1.2614, Train Accuracy: 68.81%, Val Loss: 0.5344, Val Accuracy: 83.54%


Epoch 2/20, Train: 100%|██████████| 1153/1153 [23:26<00:00,  1.22s/it, loss=0.458, accuracy=85.1]


...

Epoch 19/20, Train: 100%|██████████| 1153/1153 [24:12<00:00,  1.26s/it, loss=0.0345, accuracy=98.8]


Epoch [19/20], Train Loss: 0.0345, Train Accuracy: 98.82%, Val Loss: 0.6369, Val Accuracy: 86.72%


Epoch 20/20, Train: 100%|██████████| 1153/1153 [23:47<00:00,  1.24s/it, loss=0.0281, accuracy=99.1]


Epoch [20/20], Train Loss: 0.0281, Train Accuracy: 99.12%, Val Loss: 0.6821, Val Accuracy: 86.23%
Model saved as model.pth
Test Loss: 0.7039, Test Accuracy: 86.07%


ValueError: All arrays must be of the same length